# Learning to use Gym, PyTorch and StableBaselines3 for Reinforcement learning
Very simple notebook for learning how to use the three tools mentioned above for reinforcement learning. I might even throw in Weights & Biases if I'm feeling lucky and then eventually move onto using MuJoCo for better physics simulation. There is a lot to do here so need to be ready for a lot of fighting
## To-Do List
- [X] Fix the error where no `torch.tensor` is being pased
- [ ] Find out why the module is trying to concat a $1\times 512$ matrix with a $4\times 32$ matrix
- [ ] Run the simulation and try and get a decent model
- [ ] Evaluate the model on a better set of variables
- [ ] Trasnfer to the GPU (somehow? Don't know if CUDA is installed)

In [1]:
import numpy as np
import sklearn as sk
import matplotlib as plt
from tqdm import tqdm
import gym
import matplotlib.pyplot as plt

MAX_EPISODES = 20
MAX_ITERATIONS = 100

gym.__version__


'0.21.0'

In [49]:
# Start with just the simple cartpole problem

env = gym.make('CartPole-v1')
env.reset()

for idx in range(MAX_EPISODES):
    observation = env.reset()
    for t in range(MAX_ITERATIONS):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        
        if done:
            print(observation)
            print(f"Epsiode finished after {t+1} timesteps")
            break
env.close()

[ 0.20911957  1.5332924  -0.23903894 -2.4424908 ]
Epsiode finished after 12 timesteps
[ 0.10150199  0.00537551 -0.22257948 -0.526837  ]
Epsiode finished after 18 timesteps
[ 0.086514    0.5725944  -0.21280129 -1.2218453 ]
Epsiode finished after 11 timesteps
[ 0.14757447  0.22970277 -0.21639788 -0.7160952 ]
Epsiode finished after 17 timesteps
[ 0.2127624   1.7960484  -0.21334131 -2.6037984 ]
Epsiode finished after 25 timesteps
[-0.07425907 -1.1834116   0.22561729  2.05927   ]
Epsiode finished after 10 timesteps
[ 0.07157841  0.5511098  -0.20987761 -1.2419422 ]
Epsiode finished after 11 timesteps
[ 0.3325854   0.7829686  -0.21491194 -0.82722205]
Epsiode finished after 36 timesteps
[ 0.08855662  0.7742765  -0.2172736  -1.4721656 ]
Epsiode finished after 10 timesteps
[ 0.11313638  1.0192634  -0.21286653 -1.6693164 ]
Epsiode finished after 13 timesteps
[-0.18807949 -1.1724463   0.23419248  1.8829023 ]
Epsiode finished after 30 timesteps
[-0.11460693 -1.5173315   0.22988546  2.5211046 ]
Epsi

In [3]:
"""
Information regarding environment:

Observation space:
(4,) array with elements: [position, velocity, angle, angular velocity]

Action space:
(1,) array that is in the range {0,1} (DISCRETE)
"""

import math 
import random
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


device = torch.device("cpu")
print(torch.__version__)

1.11.0


The idea is to convert the work in the "Hands-On ML" book to work in PyTorch. I'd rather work in PyTorch simply for my own work. For deep Q-learning, we get a function $Q^*:\textit{State}\times\textit{Action}\rightarrow \mathbb{R}$ which gives us the return for a specific action in a state. We want to maximise this: $\pi^*(s)=\underset{a}{\mbox{argmax }}Q^*(s, a)$

In [4]:
input_shape = 4 # Input shape is the observations of the cartpole [pos, vel, ang, ang_vel]
output_shape = 2 # Output shape is the action space size {-1,1}

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    """
    Replay buffer used to store the previous steps taken in the training algorithm. Uses the deque function.
    (Could change to using the Reverb library from DeepMind)
    """

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
# Build a DQN model in PyTorch. This is done using a class function to create a model parameters

class DQN(nn.Module):
    """
    Very basic network that has all linear layers with an input of 4 (the observations) and an output of 2 (the actions)
    """
    
    def __init__(self, inputs, output):
        super(DQN, self).__init__()
        self.l_input = nn.Linear(inputs, 32)
        self.hidden_1 = nn.Linear(32, 32)
        self.l_output = nn.Linear(32, output)
        
        
    def forward(self, x):
        x = self.l_input(x)
        x = F.relu(x)
        x = self.hidden_1(x)
        x = F.relu(x)
        return self.l_output(x)
    
net = DQN(input_shape, output_shape)
net

DQN(
  (l_input): Linear(in_features=4, out_features=32, bias=True)
  (hidden_1): Linear(in_features=32, out_features=32, bias=True)
  (l_output): Linear(in_features=32, out_features=2, bias=True)
)

In [6]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

policy_net = DQN(input_shape, output_shape).to(device)
target_net = DQN(input_shape, output_shape).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DQN(
  (l_input): Linear(in_features=4, out_features=32, bias=True)
  (hidden_1): Linear(in_features=32, out_features=32, bias=True)
  (l_output): Linear(in_features=32, out_features=2, bias=True)
)

For this next part, we need to setup the optimiser for our model along with the function that determines taking a new step in the next direction. These functions will be adapted from the section in the "Hands-On ML" book, but using PyTorch for better future proofing

In [7]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) *  math.exp(-1*steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(0)[1].view(1,1)
    else:
        return torch.tensor([[random.randrange(output_shape)]], device=device, dtype=torch.long)
    
def optimise_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward = torch.cat(batch.reward)
    
    # Compute Q(s_t, a) for the model. 
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all the next states
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Compute the Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    
    optimizer.zero_grad()
    loss.backward()
    
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1,1)
        
    # Step the optimiser
    optimizer.step()

Final step is the actual training loop. This is the part that causes problems, so need to be careful how we do this. The observation comes in the form of a `nparray` whereas we want it in the form of a `torch.tensor`. This conversion has to be done before the model is called as otherwise it doesn't work

In [8]:
NUM_EPISODES = 5 # DOES NOT WORK AT NUMBERS GREATER THAN THIS
NUM_ITERATIONS = 200

episode_durations = []
steps_done = 0

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

"""
TODO:
Need to find out why it is not a torch tensor. Do I need to convert everything to tensors to stop the error?
UPDATE 25/05/2022@17:00 - It seems I do need to change everything to tensors to get the error to go away, but it seems that the memory isn't clearing
"""
for i_episode in range(NUM_EPISODES):
    
    print(f"Episode Number: {i_episode}")
    
    # Reset the env at the beginning of the episode
    obs = env.reset()
    
    for idx in tqdm(range(NUM_ITERATIONS)):
        # Convert the state and get the action
        state = torch.from_numpy(obs)
        action = select_action(state)
        
        # Step the environment with the chosen action
        state_obs, reward, done, info = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        
        # Check to see if the env is done or not
        if not done:
            next_state = torch.from_numpy(state_obs)
        else:
            next_state = None
        
        # Add this information to the buffer
        memory.push(state, action, next_state, reward)
        
        # Move onto the next state and optimise the model
        obs = state_obs
        optimise_model()
        
        if done:
            episode_durations.append(idx + 1)
            break;
    if i_episode & TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
print("Finished training")             

Episode Number: 0


  6%|▌         | 11/200 [00:00<00:00, 3359.60it/s]


Episode Number: 1


  9%|▉         | 18/200 [00:00<00:00, 5026.80it/s]


Episode Number: 2


...

Episode Number: 3


 14%|█▎        | 27/200 [00:00<00:00, 5266.78it/s]


Episode Number: 4


  4%|▍         | 9/200 [00:00<00:00, 2895.29it/s]

Finished training


In [9]:
env.reset()

for idx in range(MAX_EPISODES):
    obs = env.reset()
    for t in range(MAX_ITERATIONS):
        env.render()
        state = torch.from_numpy(obs)
        action = select_action(state)
        observation, reward, done, info = env.step(action.item())
        
        if done:
            print(observation)
            print(f"Epsiode finished after {t+1} timesteps")
            break
env.close()

[-0.14977871 -0.9963823   0.21811153  1.5715076 ]
Epsiode finished after 15 timesteps
[-0.09403744 -1.3513973   0.22782877  2.3029728 ]
Epsiode finished after 11 timesteps
[-0.18558191 -1.9796238   0.2556672   3.062982  ]
Epsiode finished after 10 timesteps
[-0.17081837 -1.2310686   0.22750783  2.0023    ]
Epsiode finished after 20 timesteps
[-0.10787734 -1.7571759   0.2162155   2.7545035 ]
Epsiode finished after 11 timesteps
[-0.18571898 -1.2154477   0.2345967   1.9286724 ]
Epsiode finished after 12 timesteps
[-0.08735792 -1.2025477   0.20997773  1.9508758 ]
Epsiode finished after 8 timesteps
[-0.1824925  -1.5907183   0.24729128  2.607066  ]
Epsiode finished after 10 timesteps
[-0.19759926 -1.3469483   0.22220594  1.8702506 ]
Epsiode finished after 21 timesteps
[-0.11023394 -1.6160027   0.20995219  2.5362864 ]
Epsiode finished after 14 timesteps
[-0.19441117 -1.7668047   0.23708229  2.6517723 ]
Epsiode finished after 21 timesteps
[-0.19800596 -1.9738538   0.21894585  3.008807  ]
Epsio

In [10]:
memory

In [43]:
transitions = memory.sample(5)
batch = Transition(*zip(*transitions))
non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                        batch.next_state)), device=device, dtype=torch.bool)
non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

state_batch = torch.cat(batch.state)
action_batch = torch.cat(batch.action)
reward = torch.cat(batch.reward)
action_batch, state_batch

(tensor([[1],
         [0],
         [1],
         [0],
         [0]]),
 tensor([-0.1631, -1.3515,  0.2068,  2.1994, -0.1400, -1.1550,  0.1696,  1.8592,
          0.0347, -0.1968,  0.0284,  0.2738, -0.1193, -1.7190,  0.1946,  2.7024,
          0.0202,  0.0236, -0.0414,  0.0214]))

In [48]:
policy_net(state_batch.view(-1, 4)).gather(1, action_batch)

tensor([[-0.0145],
        [ 0.1078],
        [-0.0530],
        [ 0.0906],
        [ 0.1185]], grad_fn=<GatherBackward0>)

In [24]:
torch.cat(batch.state,-1)

tensor([-0.0174, -0.5916,  0.0371,  0.8774, -0.0648, -0.9849,  0.1091,  1.5368,
         0.0295, -0.0054,  0.0432,  0.0629,  0.0054, -0.5945,  0.0876,  1.0268,
         0.0346,  0.3871,  0.0279, -0.5734])

In [26]:
batch.state[1]

tensor([-0.0648, -0.9849,  0.1091,  1.5368])

In [28]:
len(batch.state)

5

In [40]:
type(torch.cat(batch.state).view(-1, 4))

torch.Tensor

In [46]:
state_batch.view(-1,4)

tensor([[-0.1631, -1.3515,  0.2068,  2.1994],
        [-0.1400, -1.1550,  0.1696,  1.8592],
        [ 0.0347, -0.1968,  0.0284,  0.2738],
        [-0.1193, -1.7190,  0.1946,  2.7024],
        [ 0.0202,  0.0236, -0.0414,  0.0214]])